<a href="https://colab.research.google.com/github/MariuszGrecki/Machine_learning_tutorial/blob/main/supervised/01_basics/03_feature_extraction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Ekstrakcja cech**

In [1]:
import numpy as np
import pandas as pd
import sklearn

sklearn.__version__

'1.2.2'

In [3]:
#funkcja pobiera dane Amazon tickety można znaleźć w bibliotece pandas web.DataReader
def fetch_financial_data(company="AMZN"):
  """
  This functions fetches stock market quotations
  """
  import pandas_datareader.data as web
  return web.DataReader(name=company, data_source='stooq')

dr_raw = fetch_financial_data()
#head() wyświetla tylko 5 pierwszych
dr_raw.head()

,Open,High,Low,Close,Volume
Date,,,,,
2024-05-17,183.76,185.3000,183.35,184.70,33175655
2024-05-16,185.60,187.3100,183.46,183.63,38834450
2024-05-15,185.97,186.7193,182.73,185.99,75459927
2024-05-14,183.82,187.7200,183.45,187.07,38698155
2024-05-13,188.00,188.3100,185.36,186.57,24898613


In [4]:
#utworzenie kopii danych
df = dr_raw.copy()
df = df[:5]
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 5 entries, 2024-05-17 to 2024-05-13
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Open    5 non-null      float64
 1   High    5 non-null      float64
 2   Low     5 non-null      float64
 3   Close   5 non-null      float64
 4   Volume  5 non-null      int64  
dtypes: float64(4), int64(1)
memory usage: 240.0 bytes


In [5]:
#generowanie nowyzh zmiennych z indeksu
df['day'] = df.index.day
df['month'] = df.index.month
df['year'] = df.index.year
df

,Open,High,Low,Close,Volume,day,month,year
Date,,,,,,,,
2024-05-17,183.76,185.3000,183.35,184.70,33175655,17,5,2024
2024-05-16,185.60,187.3100,183.46,183.63,38834450,16,5,2024
2024-05-15,185.97,186.7193,182.73,185.99,75459927,15,5,2024
2024-05-14,183.82,187.7200,183.45,187.07,38698155,14,5,2024
2024-05-13,188.00,188.3100,185.36,186.57,24898613,13,5,2024


**Dyskretyzacja zmiennej ciągłej**

In [6]:
#dyskretyzacja zmiennej ciągłej, przedtawić jako zmienną kategoryczną
df = pd.DataFrame(data={'height': [175., 178.5, 185., 191., 184.5, 183., 168.]})
df

,height
0,175.0
1,178.5
2,185.0
3,191.0
4,184.5
5,183.0
6,168.0


In [8]:
# stworzy kolumnę height_cat i przydzieli height do odpowiednich przedziałów
df['height_cat'] = pd.cut(x=df.height, bins=3)
df

,height,height_cat
0,175.0,"(167.977, 175.667]"
1,178.5,"(175.667, 183.333]"
2,185.0,"(183.333, 191.0]"
3,191.0,"(183.333, 191.0]"
4,184.5,"(183.333, 191.0]"
5,183.0,"(175.667, 183.333]"
6,168.0,"(167.977, 175.667]"


In [9]:
#stworzy nowa kolumnę i przydzieli height ale do naszych własnych przedziałów
df['height_cat'] = pd.cut(x=df.height, bins=(160, 175, 180, 195))
df

,height,height_cat
0,175.0,"(160, 175]"
1,178.5,"(175, 180]"
2,185.0,"(180, 195]"
3,191.0,"(180, 195]"
4,184.5,"(180, 195]"
5,183.0,"(180, 195]"
6,168.0,"(160, 175]"


In [10]:
#stworzy nowa kolumnę i przydzieli height ale do naszych własnych przedziałów w tym wypadu tekstowych
df['height_cat'] = pd.cut(x=df.height, bins=(160, 175, 180, 195), labels=['small', 'medium', 'high'])
df

,height,height_cat
0,175.0,small
1,178.5,medium
2,185.0,high
3,191.0,high
4,184.5,high
5,183.0,high
6,168.0,small


In [12]:
#
pd.get_dummies(df, drop_first=True, prefix='height', dtype=int)

,height,height_medium,height_high
0,175.0,0,0
1,178.5,1,0
2,185.0,0,1
3,191.0,0,1
4,184.5,0,1
5,183.0,0,1
6,168.0,0,0


In [13]:
df = pd.DataFrame(data={'lang': [['PL', 'ENG'], ['GER', 'ENG', 'PL', 'FRA'], ['RUS']]})
df

,lang
0,"[PL, ENG]"
1,"[GER, ENG, PL, FRA]"
2,[RUS]


In [14]:
#apply pozwala nam zastosować funkcję w  tym wypadku len
df['lang_number'] = df['lang'].apply(len)
df

,lang,lang_number
0,"[PL, ENG]",2
1,"[GER, ENG, PL, FRA]",4
2,[RUS],1


In [15]:
#stworzenie kolumny z flagą która określa czy język PL jest w rekordzie
df['PL_flag'] = df['lang'].apply(lambda x: 1 if 'PL' in x else 0)
df

,lang,lang_number,PL_flag
0,"[PL, ENG]",2,1
1,"[GER, ENG, PL, FRA]",4,1
2,[RUS],1,0


In [16]:
df = pd.DataFrame(data={'website': ['wp.pl', 'onet.pl', 'google.com']})
df

,website
0,wp.pl
1,onet.pl
2,google.com


In [17]:
#dzielimy kolumne website na dwie kolumny w jednej jest np.: wp w drugiej pl
df.website.str.split('.', expand=True)

,0,1
0,wp,pl
1,onet,pl
2,google,com


In [18]:
#tutaj dzielimy kolumne website przy czym wartości uzyskane przypisujemy do kolumn z naszymi nazwami
new = df.website.str.split('.', expand=True)
df['portal'] = new[0]
df['extension'] = new[1]
df

,website,portal,extension
0,wp.pl,wp,pl
1,onet.pl,onet,pl
2,google.com,google,com
